# Multiple pulses

This notebook will illustrate how to use multiple pulses in a model.

In [ ]:
import scipp as sc
import tof

Hz = sc.Unit('Hz')
deg = sc.Unit('deg')
meter = sc.Unit('m')

## Create a source with 3 pulses

We first create an ESS source with 3 pulses, each containing 1 million neutrons, using the `pulses` argument.

In [ ]:
source = tof.Source(facility='ess', neutrons=1_000_000, pulses=3)
source

In [ ]:
source.plot()

## Chopper and detector set-up

We create two WFM choppers, and two frame-overlap choppers, and a single detector 32 meters from the source.

In [ ]:
choppers = [
    tof.Chopper(
        frequency=-70.0 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[98.71, 155.49, 208.26, 257.32, 302.91, 345.3],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[109.7, 170.79, 227.56, 280.33, 329.37, 375.0],
            unit='deg',
        ),
        phase=47.10 * deg,
        distance=6.6 * meter,
        name="WFM1",
    ),
    tof.Chopper(
        frequency=-70 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[80.04, 141.1, 197.88, 250.67, 299.73, 345.0],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[91.03, 156.4, 217.18, 269.97, 322.74, 375.0],
            unit='deg',
        ),
        phase=76.76 * deg,
        distance=7.1 * meter,
        name="WFM2",
    ),
    tof.Chopper(
        frequency=-56 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[74.6, 139.6, 194.3, 245.3, 294.8, 347.2],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[95.2, 162.8, 216.1, 263.1, 310.5, 371.6],
            unit='deg',
        ),
        phase=62.40 * deg,
        distance=8.8 * meter,
        name="Frame-overlap 1",
    ),
    tof.Chopper(
        frequency=-28 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[98.0, 154.0, 206.8, 254.0, 299.0, 344.65],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[134.6, 190.06, 237.01, 280.88, 323.56, 373.76],
            unit='deg',
        ),
        phase=12.27 * deg,
        distance=15.9 * meter,
        name="Frame-overlap 2",
    ),
]

detectors = [
    tof.Detector(distance=32.0 * meter, name='detector'),
]

## Results

We combine the `source`, `choppers`, and `detectors` into our `model`,
and then use the `.run()` method to execute the ray-tracing simulation.

In [ ]:
model = tof.Model(source=source, choppers=choppers, detectors=detectors)
res = model.run()
res

In [ ]:
res.plot(max_rays=10000)

The time-distance diagram reveals that a small number of long-wavelength neutrons from one pulse are polluting the counts detected by the detector for the next pulse (red lines).

The overlap is also visible when plotting the data seen by the detector,
even though the number of polluting neutrons is very small (the tails of each pulse are almost flat).

In [ ]:
res.detectors['detector'].tofs.plot()

To try and obtain as clean as possible of a detector signal,
we include an additional chopper in the beamline to remove pulse overlap:

In [ ]:
pol = tof.Chopper(
    frequency=-14 * Hz,
    open=sc.array(
        dims=['cutout'],
        values=[50.0],
        unit='deg',
    ),
    close=sc.array(
        dims=['cutout'],
        values=[240.0],
        unit='deg',
    ),
    phase=0 * deg,
    distance=18 * meter,
    name="Pulse-overlap",
)

model.add(pol)
res = model.run()
res.plot(max_rays=5000)

We can now see that the pulses do not overlap at the detector,
and this is confirmed in the detector plot

In [ ]:
res.detectors['detector'].tofs.plot()

## Data inspection

The detector and the chopper readings are a data group with one entry per pulse 

In [ ]:
res.detectors['detector'].tofs.visible.data

It is possible to inspect just a single pulse using the usual slicing notation `[0]` for an array:

In [ ]:
res.detectors['detector'].tofs.visible[0].data

In [ ]:
res.detectors['detector'].tofs.visible[0].plot()

and the same is available one level above on the `tofs` property,
in which case both visible and blocked neutrons for the given pulse are returned:

In [ ]:
res.choppers['WFM1'].tofs[0].plot()